In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [26]:

# Sample data: user ratings for web series
data = {
    'User': ['Naresh', 'Naresh', 'Naresh', 'Pavan', 'Pavan', 'Siddu', 'Siddu', 'Siddu', 'Karthik', 'Karthik'],
    'webseries': ['Queen of tears', 'Squid Game', 'Money Heist', 'Queen of tears', 'Stranger Things', 'Squid Game', 'Money Heist', 'Stranger Things', 'Queen of tears', 'Money Heist'],
    'Rating': [5, 3, 4, 4, 2, 5, 4, 1, 4, 5]
}

# Create DataFrame
df = pd.DataFrame(data)

def add_user_ratings(user, ratings):
    """
    Add the new user's ratings to the DataFrame.
    """
    global df
    new_data = {'User': [user] * len(ratings), 'webseries': list(ratings.keys()), 'Rating': list(ratings.values())}
    new_df = pd.DataFrame(new_data)
    df = pd.concat([df, new_df], ignore_index=True)

def recommend_webseries(user, num_recommendations=2):
    # Pivot table to create user-item matrix
    user_webseries_matrix = df.pivot_table(index='User', columns='webseries', values='Rating').fillna(0)

    # Calculate cosine similarity between users
    user_similarity = cosine_similarity(user_webseries_matrix)

    # Create a DataFrame for user similarity
    user_similarity_df = pd.DataFrame(user_similarity, index=user_webseries_matrix.index, columns=user_webseries_matrix.index)

    if user not in user_similarity_df.index:
        return "User not found."

    # Get the similarity scores for the specified user
    user_similarities = user_similarity_df[user]

    # Get the web series rated by the specified user
    user_ratings = user_webseries_matrix.loc[user]

    # Calculate weighted sum of ratings for each web series
    weighted_sum = np.dot(user_similarity_df[user].values, user_webseries_matrix.fillna(0).values)

    # Normalize by the sum of similarity scores
    sum_of_similarities = user_similarity_df[user].sum()
    weighted_average_ratings = weighted_sum / sum_of_similarities

    # Create a DataFrame for weighted average ratings
    weighted_ratings_df = pd.DataFrame(weighted_average_ratings, index=user_webseries_matrix.columns, columns=['Weighted_Rating'])

    # Filter out web series already rated by the user
    weighted_ratings_df = weighted_ratings_df.loc[~weighted_ratings_df.index.isin(user_ratings[user_ratings > 0].index)]

    # Sort by weighted rating and get top N recommendations
    recommendations = weighted_ratings_df.sort_values(by='Weighted_Rating', ascending=False).head(num_recommendations)

    return recommendations.index.tolist()

def get_user_input():
    user = input("Enter your user ID: ")
    print("Please enter your ratings for the following web series (if you haven't watched a web series, just press Enter):")

    webseries_list = ['Queen of tears', 'Squid Game', 'Money Heist', 'Stranger Things']
    ratings = {}

    for webseries in webseries_list:
        rating = input(f"Rating for {webseries} (0-5): ")
        if rating:
            ratings[webseries] = float(rating)

    return user, ratings

def chat():
    print("Welcome to the Web Series Recommendation System Using AI!")
    user, ratings = get_user_input()
    add_user_ratings(user, ratings)

    recommended_webseries = recommend_webseries(user)
    print(f"Recommended web series for {user}: {recommended_webseries}")

# Run the chat function
chat()


Welcome to the Web Series Recommendation System Using AI!
Enter your user ID: Karthik
Please enter your ratings for the following web series (if you haven't watched a web series, just press Enter):
Rating for Queen of tears (0-5): 5
Rating for Squid Game (0-5): 
Rating for Money Heist (0-5): 2
Rating for Stranger Things (0-5): 4
Recommended web series for Karthik: ['Squid Game']
